<a href="https://colab.research.google.com/github/ouharu/Machine_Learning_Study_Path/blob/master/ae_oliveface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import os
import scipy.io

import numpy as np
import tensorflow as tf
from tensorflow import keras

#from acc import acc
from sklearn.metrics import normalized_mutual_info_score as nmi

from sklearn.preprocessing import StandardScaler

# from sklearn.cluster import MiniBatchKMeans # for n > 10k
from sklearn.cluster import KMeans
# from sklearn.manifold import SpectralEmbedding

from sklearn.datasets import fetch_olivetti_faces # at&t dataset

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [123]:
import numpy as np
from sklearn import metrics
from scipy.optimize import linear_sum_assignment

# https://stackoverflow.com/questions/34047540/python-clustering-purity-metric

def acc(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)

    # Find optimal one-to-one mapping between cluster labels and true labels
    row_ind, col_ind = linear_sum_assignment(-contingency_matrix)

    # Return cluster accuracy
    return contingency_matrix[row_ind, col_ind].sum() / np.sum(contingency_matrix)

In [124]:
olivetti_face = fetch_olivetti_faces(return_X_y=False, shuffle=False,
                                     random_state=0)

In [125]:
Xd = olivetti_face.data
images = olivetti_face.images
y = olivetti_face.target

In [126]:
X = StandardScaler().fit_transform(Xd)

In [127]:
tf.random.set_seed(0)
np.random.seed(0)

act = 'selu'
kinit = 'lecun_normal'
code = 40


encoder = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[4096]),
    #keras.layers.Dropout(.5),
#     keras.layers.Dense(2048,activation=act,kernel_initializer=kinit),
    #keras.layers.BatchNormalization(),
    keras.layers.Dense(1024,activation=act,kernel_initializer=kinit),
    keras.layers.BatchNormalization(),
    # keras.layers.Dense(512,activation=act,kernel_initializer=kinit),
    # keras.layers.BatchNormalization(),
    keras.layers.Dense(256,activation=act,kernel_initializer=kinit),
    keras.layers.BatchNormalization(),
    # keras.layers.Dense(512,activation=act,kernel_initializer=kinit),
    # keras.layers.BatchNormalization(),
    # keras.layers.Dense(128,activation=act,kernel_initializer=kinit),
    # keras.layers.BatchNormalization(),
    
    keras.layers.Dense(code),
])
decoder = keras.models.Sequential([
    keras.layers.Dense(256, input_shape=[code,], activation=act,kernel_initializer=kinit),
    keras.layers.BatchNormalization(),
    # keras.layers.Dense(512,activation=act,kernel_initializer=kinit),
    # keras.layers.BatchNormalization(),
#     keras.layers.Dense(256,activation=act,kernel_initializer=kinit),
#     keras.layers.BatchNormalization(),
    # keras.layers.Dense(512,activation=act,kernel_initializer=kinit),
    # keras.layers.BatchNormalization(),
    keras.layers.Dense(1024,activation=act,kernel_initializer=kinit),
    keras.layers.BatchNormalization(),
    
#     keras.layers.Dense(2048,activation=act,kernel_initializer=kinit),
#     keras.layers.BatchNormalization(),
    keras.layers.Dense(4096,),
    
])


In [128]:
stacked_ae = keras.models.Sequential([encoder, decoder])

stacked_ae.compile(loss="MSE",
                   optimizer=keras.optimizers.SGD(lr=1.5,momentum=.9)) # .1=lecun_normal

history = stacked_ae.fit(X, X, 
                         epochs=400,
                         batch_size=32,
                        )
#stacked_ae.save("sae.h5")

Epoch 1/400
13/13 [==============================] - 1s 5ms/step - loss: 1.0246
Epoch 2/400
13/13 [==============================] - 0s 4ms/step - loss: 0.5466
Epoch 3/400
13/13 [==============================] - 0s 5ms/step - loss: 0.4751
Epoch 4/400
13/13 [==============================] - 0s 5ms/step - loss: 0.4223
Epoch 5/400
13/13 [==============================] - 0s 4ms/step - loss: 0.4038
Epoch 6/400
13/13 [==============================] - 0s 4ms/step - loss: 0.3590
Epoch 7/400
13/13 [==============================] - 0s 4ms/step - loss: 0.3297
Epoch 8/400
13/13 [==============================] - 0s 4ms/step - loss: 0.3230
Epoch 9/400
13/13 [==============================] - 0s 5ms/step - loss: 0.3002
Epoch 10/400
13/13 [==============================] - 0s 4ms/step - loss: 0.2904
Epoch 11/400
13/13 [==============================] - 0s 4ms/step - loss: 0.2785
Epoch 12/400
13/13 [==============================] - 0s 4ms/step - loss: 0.2844
Epoch 13/400
13/13 [=================

In [129]:
ae_acc = []
ae_nmi = []

for i in range(10):
    arr = np.random.permutation(len(Xd))
    
    Xnew = encoder.predict(X[arr])
    #print(Xnew.shape)
    pred = KMeans(n_clusters=40,).fit_predict(Xnew)
    ae_acc.append(acc(y[arr],pred))
    ae_nmi.append(nmi(y[arr],pred))


In [130]:
ae_acc_nmi = np.vstack((ae_acc,ae_nmi))
ae_acc_nmi

array([[0.7225    , 0.6675    , 0.6625    , 0.7025    , 0.7175    ,
        0.69      , 0.65      , 0.665     , 0.69      , 0.6625    ],
       [0.84729412, 0.82885784, 0.81389593, 0.83359114, 0.83470821,
        0.82585366, 0.79683107, 0.81624857, 0.82722553, 0.82218268]])

In [131]:
np.mean(ae_acc_nmi,axis=1)

array([0.683     , 0.82466887])

In [132]:
np.std(ae_acc_nmi,axis=1)

array([0.02392175, 0.01298409])